Packages:

In [ ]:
import glob
import os
import warnings
 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from pydicom.data import get_testdata_files
from sklearn.model_selection import train_test_split

from IPython.display import Markdown, display

 # Import functions from the module
import importlib
import help_files._0_definitions 
# import  help_files._1_visuals_script
# import  help_files._01_load_data
 # Reload the module to apply the changes to the script
importlib.reload(help_files._0_definitions)
 
# importlib.reload(help_files._01_load_data)
 
# Group by 'condition', 'level', and 'severity' and count occurrences
from help_files._0_definitions import count_severity_by_condition_level 
# Define the path
from pathlib import Path

pd.set_option("display.width", 1000)  # Set a large width to prevent line wrapping
 

In [ ]:
### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_definitions.py") as file:
    exec(file.read())
    ### In definitions are all the functions that are used in the notebook and globals
with open("help_files/_0_run_definitions.py") as file:
    exec(file.read())

In [ ]:
# loading data
file_names = ["train_df_3_cat.csv", "test_df_3_cat.csv"]
# Load the data from the CSV files
dataframes = [pd.read_csv(data_path_vor / file_name) for file_name in file_names]
# Unpack the dataframes into separate variables
train_df, test_df = dataframes

print("DataFrames have been loaded successfully.")



In [ ]:
# Distribution of severity in test_df
test_severity_distribution = test_df['severity'].value_counts(normalize=True)
print("\nTest DataFrame Severity Distribution:")
print(test_severity_distribution)

# Create a variable for test_df data frame giving the number of rows for each study_id

# Calculate accuracy as severity 0 divided by the sum of severity 1 and 2
severity_0 = test_severity_distribution.get(0, 0)
severity_1 = test_severity_distribution.get(1, 0)
severity_2 = test_severity_distribution.get(2, 0)
accuracy = severity_0 / (severity_0 + severity_1 + severity_2)  
print(f"\nAccuracy: {accuracy:.4f}")

In [ ]:
# Calculate the normalized severity distribution (proportions for each severity level)
test_severity_distribution = test_df['severity'].value_counts(normalize=True)
print("\nTest DataFrame Severity Distribution:")
print(test_severity_distribution)

# Extract proportions for each severity level (if not present, default to 0)
severity_0 = test_severity_distribution.get(0, 0)
severity_1 = test_severity_distribution.get(1, 0)
severity_2 = test_severity_distribution.get(2, 0)

# Baseline accuracy is simply the proportion of the majority class
baseline_accuracy = max(severity_0, severity_1, severity_2)

print(f"\nBaseline Accuracy: {baseline_accuracy:.2%}")


In [ ]:
test_severity_distribution = test_df['severity'].value_counts()

# Get the total number of samples in the test set
total_samples = test_severity_distribution.sum()

# Find the count of the majority class
majority_class_count = test_severity_distribution.max()

# Calculate baseline accuracy
baseline_accuracy = majority_class_count / total_samples if total_samples > 0 else 0

print(f"Baseline Accuracy: {baseline_accuracy:.2%}")

In [ ]:
train_df

In [ ]:
# # end sample or small sample    
if whole_data_set == False:
    print("Using the whole data set")
else:
    train_df = train_df.sample(n=50, random_state=RSEED)
    test_df = test_df.sample(n=300, random_state=RSEED)
    display(Markdown('<span style="color:red"> this is a small sample : 48692</span>'))
    train_df.dtypes
train_df['condition'] = train_df['condition'].astype('category').cat.codes
train_data = train_df

In [ ]:
 ss

In [ ]:
# print(train_df.to_string(index=False, header=True)) 

In [ ]:
print(train_df['severity'].unique())

In [ ]:
train_df['condition'] = train_df['condition'].astype('category').cat.codes
train_data = train_df

In [ ]:
import pydicom
import pandas as pd
import torch
import random
import numpy as np
import torchvision.models as models
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import cv2
from PIL import Image, ImageDraw
import mlflow
import os
import matplotlib.pyplot as plt

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Define the transform with augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomRotation(4),       # Randomly rotate the image by ±4 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to create a bounding box around (x, y) coordinates
def create_bounding_box(image, x, y, box_size=50):
    image = image.convert('RGB')  # Convert to RGB
    draw = ImageDraw.Draw(image)

    half_box = box_size // 2
    left = x - half_box
    top = y - half_box
    right = x + half_box
    bottom = y + half_box
    
    draw.rectangle([left, top, right, bottom], outline="red", width=3)
    
    return image, left, top, right, bottom

# Function to process all images and update the bounding boxes
def process_images_and_create_boxes(df, box_size=50):
    for index, row in df.iterrows():
        if pd.isna(row['x']) or pd.isna(row['y']):
            print(f"Skipping index {index} due to missing x/y coordinates")
            continue
        
        image_path = row['image_path']
        x = row['x']
        y = row['y']

        try:
            dicom_image = pydicom.dcmread(image_path)
            image = dicom_image.pixel_array.astype(float)
            image = (image / image.max() * 255).astype('uint8')

            if len(image.shape) == 2:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

            image = Image.fromarray(image)
        except Exception as e:
            print(f"Error reading DICOM image at index {index}: {e}")
            continue

        image_with_box, left, top, right, bottom = create_bounding_box(image, x, y, box_size)
        df.at[index, 'x_min'] = left
        df.at[index, 'y_min'] = top
        df.at[index, 'x_max'] = right
        df.at[index, 'y_max'] = bottom

        if index == 0:
            image_with_box.save(f"image_with_box_{index}.png")
        
        print(f"Updated bounding box for index {index}: x_min={left}, y_min={top}, x_max={right}, y_max={bottom}")

    return df

# Assuming the DataFrame `train_df` is already loaded
train_df = process_images_and_create_boxes(train_df)

# Save the updated DataFrame with bounding boxes
# train_df.to_csv('train_with_boxes.csv', index=False)

# Define MRIDataset class
class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']

        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')

        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)

        return image_tensor, torch.tensor(label).long()

    def __len__(self):
        return len(self.data)

# Create the dataset
# Remove rows where 'image_path' is NaN before filtering
train_data = train_df[train_df['image_path'].notna() & train_df['image_path'].str.endswith('.dcm')]

dataset = MRIDataset(data=train_data, transform=transform)

# Split the dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load ResNet-50 and set up for classification
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = train_df['severity'].nunique()

model = models.resnet50(weights='IMAGENET1K_V1')
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define loss functions and optimizer
learning_rate = 0.0001
criterion_cel = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
num_epochs = 20

# Early stopping parameters
diverge_count = 0
stop_threshold = 0.3
max_diverge_count = 3
patience = 5

# Start MLflow run
with mlflow.start_run():
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("num_epochs", num_epochs)
    mlflow.log_param("num_classes", num_classes)
    mlflow.log_param("model_architecture", "ResNet-50")
    mlflow.log_param("stop_threshold", stop_threshold)
    mlflow.log_param("max_diverge_count", max_diverge_count)
    mlflow.log_param("patience", patience)
    mlflow.set_tag("model_description", "ResNet-50 for 3 cat and Sagittal T2/STIR and Sagittal T1 images")

    # Example input tensor with the same shape as the model's expected input
    example_input = torch.randn(1, 3, 224, 224)  # Batch size 1, 3 color channels, 224x224 image
    best_val_loss = float('inf')
    patience_counter = 0
    diverge_count = 0

    train_losses_cel = []
    val_losses_cel = []

    for epoch in range(num_epochs):
        model.train()
        running_loss_cel_train = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss_cel = criterion_cel(outputs, labels)
            running_loss_cel_train += loss_cel.item()
            loss_cel.backward()
            optimizer.step()

        epoch_loss_cel_train = running_loss_cel_train / len(train_loader)
        train_losses_cel.append(epoch_loss_cel_train)
        mlflow.log_metric("train_loss_cel", epoch_loss_cel_train, step=epoch)

        model.eval()
        running_loss_cel_val = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss_cel = criterion_cel(outputs, labels)
                running_loss_cel_val += loss_cel.item()

        epoch_loss_cel_val = running_loss_cel_val / len(val_loader)
        val_losses_cel.append(epoch_loss_cel_val)
        mlflow.log_metric("val_loss_cel", epoch_loss_cel_val, step=epoch)

        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Train Cross Entropy Loss: {epoch_loss_cel_train:.4f}, '
              f'Validation Cross Entropy Loss: {epoch_loss_cel_val:.4f}')

        epoch_weight_path = f"model_weights_epoch_{epoch + 1}.pt"
        torch.save(model.state_dict(), epoch_weight_path)
        mlflow.log_artifact(epoch_weight_path)
        os.remove(epoch_weight_path)

        if epoch_loss_cel_val > epoch_loss_cel_train * (1 + stop_threshold):
            diverge_count += 1
            if diverge_count >= max_diverge_count:
                print(f"Early stopping at epoch {epoch + 1} due to validation loss diverging.")
                break
        else:
            diverge_count = 0

        if epoch_loss_cel_val < best_val_loss:
            best_val_loss = epoch_loss_cel_val
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1} due to no improvement in validation loss.")
                break

    # Log final model
    example_input_np = example_input.numpy()
    mlflow.pytorch.log_model(model, "final_model", input_example=example_input_np)

    # Plot and log the loss curves as artifacts
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses_cel, label='Train Cross Entropy Loss')
    plt.plot(val_losses_cel, label='Validation Cross Entropy Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Cross Entropy Loss')
    plt.legend()
    plt.savefig("cross_entropy_loss.png")
    mlflow.log_artifact("cross_entropy_loss.png")


In [ ]:
sss

Predictions

In [ ]:
import torch
import torchvision.models as models

# Step 1: Define the model architecture (same as training)
model_for_prediciton = models.resnet50()
model_for_prediciton.fc = torch.nn.Linear(model_for_prediciton.fc.in_features, 3)  # Update for your classes

# Step 2: Load the best model weights
# Adjust this path if necessary
best_weights_path = "C:/Users/HP1/Desktop/Spiced/capstone-project/mlruns/711328181724227740/deaa903e488043d19aad572a6c429479/artifacts/best_model_weights_epoch_1.pt"
model_for_prediciton.load_state_dict(torch.load(best_weights_path))

# Step 3: Set the model to evaluation mode
model_for_prediciton.eval()
model_for_prediciton.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

In [ ]:
# Replace <run_id> with your actual run ID and <path_to_model> with the artifact path used during logging
model_path = r"C:\Users\HP1\Desktop\Spiced\capstone-project\mlruns\0\632927202a334496b1846096a752df1f\artifacts\final_model"
model_for_prediciton = mlflow.pytorch.load_model(model_path)

model_for_prediciton.eval()  # Set the model to evaluation mode

Restore parameters and metrtics from mlflow

I should take prdicted probabilities and not predicted classes: output scores (logits) are converted into probabilities using the softmax function

In [ ]:
import pydicom
import pandas as pd
import torch
import random
import numpy as np
import torchvision.models as models
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import cv2
from PIL import Image, ImageDraw
import mlflow
import os
import matplotlib.pyplot as plt

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Define the transform with augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomRotation(4),       # Randomly rotate the image by ±4 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Function to create a bounding box around (x, y) coordinates
def create_bounding_box(image, x, y, box_size=50):
    image = image.convert('RGB')  # Convert to RGB
    draw = ImageDraw.Draw(image)

    half_box = box_size // 2
    left = x - half_box
    top = y - half_box
    right = x + half_box
    bottom = y + half_box
    
    draw.rectangle([left, top, right, bottom], outline="red", width=3)
    
    return image, left, top, right, bottom

# Function to process all images and update the bounding boxes
def process_images_and_create_boxes(df, box_size=50):
    for index, row in df.iterrows():
        if pd.isna(row['x']) or pd.isna(row['y']):
            print(f"Skipping index {index} due to missing x/y coordinates")
            continue
        
        image_path = row['image_path']
        x = row['x']
        y = row['y']

        try:
            dicom_image = pydicom.dcmread(image_path)
            image = dicom_image.pixel_array.astype(float)
            image = (image / image.max() * 255).astype('uint8')

            if len(image.shape) == 2:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

            image = Image.fromarray(image)
        except Exception as e:
            print(f"Error reading DICOM image at index {index}: {e}")
            continue

        image_with_box, left, top, right, bottom = create_bounding_box(image, x, y, box_size)
        df.at[index, 'x_min'] = left
        df.at[index, 'y_min'] = top
        df.at[index, 'x_max'] = right
        df.at[index, 'y_max'] = bottom

        if index == 0:
            image_with_box.save(f"image_with_box_{index}.png")
        
        print(f"Updated bounding box for index {index}: x_min={left}, y_min={top}, x_max={right}, y_max={bottom}")

    return df

# Define MRIDataset class
class MRIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        self.data['severity'] = self.data['severity'].astype(int)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        image_path = row['image_path']
        label = row['severity']

        dicom_image = pydicom.dcmread(image_path)
        image = dicom_image.pixel_array.astype(float)
        image = (image / image.max() * 255).astype('uint8')

        if len(image.shape) == 2:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

        image_tensor = self.transform(image) if self.transform else torch.from_numpy(image).permute(2, 0, 1)

        return image_tensor, torch.tensor(label).long()

    def __len__(self):
        return len(self.data)

predicting probabilities for test data

In [ ]:
test_data = test_df
# Assuming the DataFrame `train_df` is already loaded
test_df = process_images_and_create_boxes(test_df)

In [ ]:
# calculate prbabilities and probability list
test_dataset = MRIDataset(data=test_data, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Inference loop with probability extraction
results = []
probabilities_list = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)  # Ensure labels are also on the correct device
        outputs = model(images)
        probabilities = torch.softmax(outputs, dim=1)  # Calculate class probabilities
        _, predicted_classes = torch.max(outputs, 1)
        
        # Append predictions and probabilities
        results.append(predicted_classes.item())
        probabilities_list.append(probabilities.cpu().numpy())



In [ ]:
# Print predicted classes and their probabilities
for i, (pred, probs) in enumerate(zip(results, probabilities_list)):
    print(f"Test image {i}: Predicted class {pred}, Probabilities: {probs}")

# Additional code to plot the confusion matrix can stay as-is:
true_labels = []
predicted_labels = []

for images, labels in test_loader:
    images = images.to(device)
    true_labels.extend(labels.numpy())

    with torch.no_grad():
        outputs = model(images)
        _, predicted_classes = torch.max(outputs, 1)
        predicted_labels.extend(predicted_classes.cpu().numpy())

true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

print("True Labels Unique Values:", np.unique(true_labels))
print("Predicted Labels Unique Values:", np.unique(predicted_labels))


In [ ]:
# Convert the list of probabilities to a numpy array for easier manipulation
probabilities_array = np.vstack(probabilities_list)

# Ensure the length of probabilities_array matches the length of test_df
probabilities_array = probabilities_array[:len(test_df)]

# Add the probabilities to the test_df DataFrame
test_df['Probability_Class_0'] = probabilities_array[:, 0]
test_df['Probability_Class_1'] = probabilities_array[:, 1]
test_df['Probability_Class_2'] = probabilities_array[:, 2]

# Round the probabilities to 4 decimal places
test_df['Probability_Class_0'] = test_df['Probability_Class_0'].round(4)
test_df['Probability_Class_1'] = test_df['Probability_Class_1'].round(4)
test_df['Probability_Class_2'] = test_df['Probability_Class_2'].round(4)
# Add the predicted classes to the test_df DataFrame
test_df['Predicted_Class'] = results



# Display the updated DataFrame
 
test_df.head()

In [ ]:
from sklearn.metrics import confusion_matrix
# Group by severity and sum the Probability_Class_1
severity_prob_sum_all_classes = test_df.groupby('severity')[['Probability_Class_0', 'Probability_Class_1', 'Probability_Class_2']].mean()

# Print the result
severity_prob_sum_all_classes
# Use severity_prob_sum_all_classes as confusion matrix
conf_matrix_prob_df = severity_prob_sum_all_classes 
# Create a confusion matrix from severity and predicted class
conf_matrix_severity_pred = confusion_matrix(test_df['severity'], test_df['Predicted_Class'])

# Create a DataFrame from the confusion matrix
conf_matrix_severity_pred_df = pd.DataFrame(
    conf_matrix_severity_pred, 
    index=[f"Actual {i}" for i in range(len(conf_matrix_severity_pred))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix_severity_pred[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_severity_pred_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')
# Print the confusion matrix DataFrame
print(conf_matrix_prob_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_prob_df.style.background_gradient(cmap='Blues')


In [ ]:
conf_matrix_severity_pred_df.style.background_gradient(cmap='Blues')

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np

# Ensure true_labels and predicted_labels have the same length
min_length = min(len(true_labels), len(predicted_labels))
true_labels = true_labels[:min_length]
predicted_labels = predicted_labels[:min_length]

# Generate the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Create a DataFrame from the confusion matrix
conf_matrix_df = pd.DataFrame(
    conf_matrix, 
    index=[f"Actual {i}" for i in range(len(conf_matrix))], 
    columns=[f"Predicted {i}" for i in range(len(conf_matrix[0]))]
)

# Print the confusion matrix DataFrame
print(conf_matrix_df)

# Optionally, display it using a more formatted view (e.g., in Jupyter Notebook)
conf_matrix_df.style.background_gradient(cmap='Blues')


In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
# Calculate precision, recall, and accuracy using mean_probabilities_df
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")
